### NORMALIZE CREATING NEW TABLE

ta metoda polega na tworzeniu tabeli dla każdej listy

In [27]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

In [8]:
paths_test = [ '../data-test/airlines.json', 
            '../data-test/gists.json', 
            '../data-test/historical-events.json',#have to rename the same value in json (movie on  the same level)
            '../data-test/movies.json',
            '../data-test/reddit.json',#bad
            '../data-test/nasa.json'
            ]

In [9]:
paths = [ '../data/airlines.json', 
            '../data/gists.json', 
            '../data/historical-events.json',
            '../data/movies.json',
            '../data/reddit.json',
            '../data/nasa.json']

In [58]:
def make_new_json(lists, table_name):
    data = []
    for i, item in enumerate(lists[table_name]):
        if isinstance(item, list):
            for elem in item or [None]:
                data.append({"row_number": i, "value": elem})
        elif item is not None:
            data.append({"row_number": i, "value": item})
    return data

def delete_lists(df, name):
    df = pd.json_normalize(df)
    is_list = df.applymap(lambda x: isinstance(x, list)).any()
    list_columns = is_list[is_list].index.tolist()

    if list_columns:
        list_df = df[list_columns].copy()
        df = df.drop(columns=list_columns)
        df['row_number'] = range(1, len(df) + 1)

    main_table = pa.Table.from_pandas(df)
    tables = {name: main_table}

    for dropped_list in list_columns:
        new_json = make_new_json(list_df, dropped_list)
        tables[dropped_list] = delete_lists(new_json, dropped_list)

    return tables

results = []

for path in paths:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)
    results.append(delete_lists(dt, 'main'))


In [64]:
results[5]

{'main': pyarrow.Table
 name: string
 id: string
 nametype: string
 recclass: string
 mass: string
 fall: string
 year: string
 reclat: string
 reclong: string
 geolocation.type: string
 :@computed_region_cbhk_fwbd: string
 :@computed_region_nnqa_25f4: string
 row_number: int64
 ----
 name: [["Aachen","Aarhus","Abee","Acapulco","Achiras",...,"Tirupati","Tissint","Tjabe","Tjerebon","Tomakovka"]]
 id: [["1","2","6","10","370",...,"24009","54823","24011","24012","24019"]]
 nametype: [["Valid","Valid","Valid","Valid","Valid",...,"Valid","Valid","Valid","Valid","Valid"]]
 recclass: [["L5","H6","EH4","Acapulcoite","L6",...,"H6","Martian (shergottite)","H6","L5","LL6"]]
 mass: [["21","720","107000","1914","780",...,"230","7000","20000","16500","600"]]
 fall: [["Fell","Fell","Fell","Fell","Fell",...,"Fell","Fell","Fell","Fell","Fell"]]
 year: [["1880-01-01T00:00:00.000","1951-01-01T00:00:00.000","1952-01-01T00:00:00.000","1976-01-01T00:00:00.000","1902-01-01T00:00:00.000",...,"1934-01-01T00:00